### Visualización y preprocesamiento de los datos

En este notebook utilizaremos el conjunto que hemos inspeccionado y adecuado para crear unas sencillas representaciones de los datos  y posteriormente aplicar tecnica de preprocesamiento realizar un sencillo modelo que nos ayude a analizar los sentimientos descritos en las diferentes reseñas. Así, el siguiente script está dividido en los siguientes bloques:

- **BLOQUE A**: carga de datos inspeccionados.
- **BLOQUE B**: visualización. 
- **BLOQUE C**: preprocesamiento de los textos.
- **BLOQUE D**: partición del conjunto de datos en train y test.
- **BLOQUE E**: balanceo de los datos.
- **BLOQUE F**: vectorización de los conjuntos de datos.

In [ ]:
import pandas as pd
import re
import pickle
import joblib
import random

import seaborn as sns
import matplotlib.pyplot as plt 

from wordcloud import WordCloud
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


### BLOQUE A: Carga de datos
Antes de comenzar, cargaremos los datos que han sido adecuados en nuestra fase anterior de limpieza:

In [ ]:
# Carga de datos ya adecuados
df = pd.read_csv(???)

In [ ]:
# ¿Que dimensiones tiene el conjunto de datos?
df.shape

In [ ]:
# Mostramos las primeras observaciones del conjunto
df.head()

### BLOQUE B: visualización

En este bloque utilizaremos las librerias [matplotlib](https://matplotlib.org/) y [seaborn](https://seaborn.pydata.org/) para crear unas sencillas representaciones de los datos a modo general y descriptivo, mientras que  nos ayudaremos de la librería [wordcloud](https://amueller.github.io/word_cloud/) para poder crear visualizaciones acerca de los textos que vamos a analizar.

In [ ]:
# Gráfico de barras para la variable target
sns.???(x='target', palette='viridis', data=???)
plt.???('Distribución de la variable target')
plt.???('Frecuencia')
plt.???('Target')
plt.show()

In [ ]:
# Gráfico 'pie' con porcentajes para la variable objetivo 
plt.???(???, autopct="%.2f%%", labels=['HAM', 'SPAM'])
plt.show()

In [ ]:
# Histograma de la distribución de las logitudes de los SMS.
# Utilizamos los histogramas proporcionados por el propio dataframe.
df[???].hist()
plt.???('Histograma de Longitudes')
plt.???('Longitud')
plt.???('Frecuencia')
plt.show()

In [ ]:
# Distribución de la longitud por cada tipo de clase - gráfico boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x=???, y=???, data=df, palette='Set2')
plt.title('Boxplot de Longitud por Polaridad')
plt.xlabel('Sentiment')
plt.ylabel('Longitud')
plt.show()

Vamos a crear un gráfico de las palabras más comunes en los SMS para cada clase:

In [ ]:
# Filtramos los textos por sus etiquetas

# textos no spam
ham_texts = df.???[df['target']==???, 'text']

# textos spam
spam_texts = df.???[df['target']==???, 'text']

In [ ]:
# Función para poder realizar el gráfico
def wordcloud_draw(data, color, title):
    words = ' '.join(data)
    wordcloud = WordCloud(stopwords=stopwords.words('english'),
                          background_color=color,
                          width=2500,height=2000).generate(words)
    plt.imshow(wordcloud)
    plt.title(title)
    plt.axis('off')

In [ ]:
# Representamos los dos gráficos en una sola visualización
plt.figure(figsize=[20,10])
plt.subplot(1,2,1)
wordcloud_draw(???,'white','Palabras más comunes en NO SPAM')

plt.subplot(1,2,2)
wordcloud_draw(???, 'grey','Palabras más comunes en SPAM')
plt.show()

### BLOQUE C: Preprocesamiento del texto

El preprocesamiento del texto es una fase importante dentro del Procesamiento del Lenguaje Natural (NLP). El objetivo de esta fase es la de transformar el texto en crudo, de manera que sea más fácilmente consumible por los algoritmos y modelos de Machine Learning (ML) y Deep Learning (DL) a aplicar.

Esta fase consta de diferentes pasos y no son siempre los mismos. En este caso, preprocesaremos los SMS de la siguiente manera:

1. **Lower Casing**: Transformar mayúsculas a minúsculas.

2. **Eliminar Non-Alphabets**: Reemplazar todos los caracteres que no aparecen en el abecedario por un espacio.

3. **Eliminar letras consecutivas**: 3 o más letras consecutivas son reemplazadas por 2 letras (ejemplo: "Heyyyy" por "Heyy").

4. **Tokenizacíon**:  proceso de dividir un texto en unidades más pequeñas llamadas tokens (en este caso, palabras).

5. **Eliminar Stopwords**: Las Stopwords son aquellas palabras que no tienen un significado específico por si solas, por lo que pueden ser ignoradas sin sacrificar el significado de la oración (ejemplos en inglés: "the", "a").

6. **Eliminar palabras cortas**: Palabras con menos de 2 letras son eliminadas.

In [ ]:
# Función para preprocesar el texto en crudo
def preprocess(text):    

    # Definir patrones para reemplazar/eliminar.
    alphaPattern      = "[^a-zA-Z]"
    sequencePattern   = r"(.)\1\1\1*"
    seqReplacePattern = r"\1\1"    
    
    # Crear lista de stopwords
    en_stop =  set(stopwords.words('english')) - {'not','no'}  # o definirlo manualmente set(['a', 'an', 'the', 'in', 'does', 'do'])

    # Lower Casing
    text = text.lower()

    # Reemplazar non-alphabets.
    text = re.sub(alphaPattern, " ", text)

    # Reemplazar letras consecutivas.
    text = re.sub(sequencePattern, seqReplacePattern, text)
    
    # Tokenizar texto
    tokens = word_tokenize(text)

    # Eliminar stopwords
    tokens = [word for word in tokens if word not in en_stop]
    
    # Eliminar tokens con menos de dos elementos/caracteres
    tokens = [word for word in tokens if len(word)>2]
    
    return tokens

In [ ]:
# Aplicamos la función a cada una de los SMS
df['preprocess_text'] = df[???].apply(???)

In [ ]:
# Resultados del preprocesamiento: un ejemplo
print('SMS en crudo:', df.loc[0, ???])
print('SMS preprocesado:', df.loc[0, ???])

### BLOQUE D: Partición del conjunto de datos en train y test (80,20)

In [ ]:
X = df['preprocess_text']
y = df['target']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(???, ???, test_size=???, stratify=y, random_state=42)

In [ ]:
# Información acerca de los conjuntos
print('Tamaño del conjunto de entrenamiento:', ???(X_train))
print('Tamaño del conjunto de test:', ???(X_test))

### BLOQUE D: Balanceo de los datos

Si hablamos de problemas de modelado centrados en la clasificación, podemos encontrarnos con **conjuntos de datos** en las que la variable objetivo contiene clases muy **desequilibradas**, es decir, categorías con frecuencias muy diferentes.

Al entrenar un modelo de clasificación con la variable no balanceada, encontraremos algunos problemas. Esto sucede porque el patrón de datos de la clase dominante superará a los de la clase con menos frecuencia. Como una de las clases tiene una frecuencia muy alta, el modelo construido con datos desequilibrados puede presentar una precisión muy alta y aun así no predecir correctamente ninguna observación para la clase con una frecuencia más baja. Esto puede dar la falsa impresión de que el modelo funciona bien cuando en realidad no es así.

Para solucionar estos problemas podemos recurrir a una solución: **equilibrar o balancear los datos de la variable objetivo**.

Como los modelos deben funcionar en el mundo real, este balanceo solo es necesario realizarlo en el **conjunto de entrenamiento**. De esta manera, el conjunto de test sobre el que evaluaremos el rendimiento de nuestros modelos seguirá reflejando lo más fielmente posible la realidad a la que se aplicarán.

In [ ]:
# Separar conjunto de datos de entrenamiento según clases
X_train_spam = X_train[y_train == 'spam']
y_train_spam = y_train[y_train == 'spam']

X_train_no_spam = X_train[y_train == 'ham']
y_train_no_spam = y_train[y_train == 'ham']

# Submuestrear (undesampling) la clase mayoritaria
X_train_no_spam_resampled, y_train_no_spam_resampled = resample(???, 
                                                                ???,
                                                                replace=False,  # sin reemplazo
                                                                n_samples=len(???),  # igualar tamaño al de "spam"
                                                                random_state=42)

# Combinar clases balanceadas
X_train_balanced = pd.concat([X_train_spam, X_train_no_spam_resampled])
y_train_balanced = pd.concat([y_train_spam, y_train_no_spam_resampled])

In [ ]:
# Información acerca de los conjuntos tras balanceo
print('Tamaño del conjunto de entrenamiento:', len(X_train_balanced))
print('Tamaño del conjunto de test:', len(X_test))

In [ ]:
# Frequencias relativas de la variable 'target' en el conjunto de entrenamiento
round(y_train_balanced.value_counts(normalize=True), 2)

In [ ]:
# Frequencias relativas de 'target' en el conjunto de test
round(y_test.value_counts(normalize=True), 2)

### BLOQUE E: Vectorización del texto

Antes de dar el texto en input a un modelo es necesario vectorizarlo: convertir las palabras en números.

La conversión del texto en una representación númerica es uno de los pasos más importantes dentro de cualquier *pipeline* de NLP. Esta conversión resulta esencial para que las "máquinas" puedan comprender y decodificar patrones dentro de cualquier lenguaje.

Se trata de un proceso iterativo y que puede ser realizado mediante múltiples maneras o técnicas, abarcando desde las representaciones más sencillas (por ejemlo, One hot encoding) hasta otras más "inteligentes", que logran tener en cuenta las similitudes y diferencias entre ellas al basar su aprendizaje en redes neuronales (Word embeddings).

En este caso vamos a utilizar la técnica **TF-IDF (Term Frequency-Inverse Document Frequency)**. A continuación, se describen los conceptos clave:

1. **Term Frequency (TF)**:
Mide la frecuencia de un término específico en un texto.
Se calcula dividiendo el número de veces que un término aparece en un texto entre el número total de términos en el propio texto.
Cuanto más frecuente es un término, mayor es su valor de TF.

2. **Inverse Document Frequency (IDF)**:
Mide la importancia de un término en el conjunto de textos.
Se calcula tomando el logaritmo del inverso de la proporción de textos que contienen el término.
Así, los términos que aparecen en muchos textos tendrán un IDF más bajo, ya que se consideran menos informativos.

3. **TF-IDF**:
Combina TF y IDF para asignar un peso a cada término en cada texto: **TF-IDF = TF * IDF** .\
Los términos que son frecuentes en un texto pero raros en el conjunto de textos tendrán un alto valor de TF-IDF, lo que indica su importancia relativa en ese texto específico.

In [ ]:
# Vectorización de textos
# Cargamos el vectorizador
vectorizer = TfidfVectorizer() 

# fit_transform() determina qué palabras existen en el conjunto de datos y asigna un índice a cada una de ellas
X_train_vec = vectorizer.???([" ".join(tokens) for tokens in ???])

In [ ]:
# Transformamos nuevos datos (conjunto de test) en función del vocabulario aprendido anteriormente (con el conjunto de entrenamiento)
X_test_vec = vectorizer.???([" ".join(tokens) for tokens in X_test])

Vamos a ver con más detalle el objecto generado con `TfidfVectorizer`:

In [ ]:
type(X_train_vec)

 Se trata de una matriz dispersa (*sparse matrix*) en formato CSR (*Compressed Sparse Row*). Una matriz dispersa es una estructura de datos que se utiliza para almacenar matrices que tienen una gran cantidad de elementos cero.

In [ ]:
# Obtener dimensiones
num_textos, num_terminos = X_train_vec.???

print(f"Número de Textos: {num_textos}")
print(f"Número de Términos: {num_terminos}")


In [ ]:
# Escogemos 10 palabras al azar  
random.???(list(vectorizer.get_feature_names_out()), ???)

Comprobamos el valor TF-IDF asignado a algunas palabras en el primer texto:

In [ ]:
# Almacenamos los términos obtenidos mediante el objeto vectorizer para utilizarlos de manera más directa
terminos = vectorizer.get_feature_names_out()

# Primer documento como vector TF-IDF
vector_tfidf_primer_documento = X_train_vec[0]

# Crear un DataFrame para visualizar el resultado
df_terminos = pd.DataFrame(vector_tfidf_primer_documento.toarray(), columns=terminos)

In [ ]:
# Valor TF-IDF asignado a una palabra contenida en el texto
df_terminos[???]

In [ ]:
# Valor TF-IDF asignado a una palabra no presente en el texto
df_terminos[???]

### BLOQUE E: Exportación de los conjuntos preprocesados

En esta última fase exportamos y guardamos los conjuntos preprocesados en la carpeta 'data', para poderlos utilizar en las próximas tareas de modelado. 


In [ ]:
# ¿De que tipo son los conjuntos resultantes?
print(f'X_train: {???(X_train_balanced)}')
print(f'X_train_vec: {???(X_train_vec)}')
print(f'y_train: {???(y_train_balanced)}')

Vamos a guardarlos como archivos binarios utilizando **pickle**, que es una forma general de serializar objetos en Python

In [ ]:
# Guardamos los conjuntos X
with open('../data/x_train.pkl', 'wb') as file:
    pickle.dump(X_train_balanced, file)

with open('../data/x_test.pkl', 'wb') as file:
    pickle.dump(X_test, file)

In [ ]:
# Guardamos los conjuntos de textos vectorizados
with open('../data/x_train_vec.pkl', 'wb') as file:
    pickle.dump(X_train_vec, file)

with open('../data/x_test_vec.pkl', 'wb') as file:
    pickle.dump(X_test_vec, file)

In [ ]:
# Guardamos las variables objetivo
with open('../data/y_train.pkl', 'wb') as file:
    pickle.dump(y_train_balanced, file)

with open('../data/y_test.pkl', 'wb') as file:
    pickle.dump(y_test, file)

In [ ]:
# Guardamos el objeto vectorizer en un fichero 
joblib.dump(vectorizer, '../models/tfidf_vectorizer.joblib')